In [1]:
%run ../../base-notebook.ipynb

2023-07-13 16:37:05 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/contents.txt [820/820] -> "../../ext/contents.txt" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_callback.py [8266/8266] -> "../../ext/ml_callback.py" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_internal.py [3032/3032] -> "../../ext/ml_internal.py" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_io.py [2664/2664] -> "../../ext/ml_io.py" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_layer.py [12200/12200] -> "../../ext/ml_layer.py" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_plot.py [26211/26211] -> "../../ext/ml_plot.py" [1]
2023-07-13 16:37:06 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_util.py [2284/2284] -> "../../ext/ml_util.py" [1]
2023-07

In [1]:
extensions_loaded = False
if not extensions_loaded:
    %load_ext autoreload
    %load_ext tensorboard
    extensions_loaded = True

%autoreload 2
%matplotlib inline

In [51]:
import sys
import os
import random
import collections

sys.path.append("../../ext")

import ext.ml_callback as mlc
import ext.ml_io as mli
import ext.ml_layer as mll
import ext.ml_plot as mlp
import ext.ml_util as mlu
import ext.ml_data as mld
import ext.ml_view as mlv
import ext.ml_analyze as mla
import ext.ml_shell as mls

import tensorflow as tf
import tensorflow_text as tf_text
import tensorflow_datasets as tfds
import tensorflow_hub as hub

import pandas as pd

import opendatasets as od

from keras import Model
from keras import optimizers
from keras import layers
from keras import losses
from keras import utils

# StackOverfloat
## Download the StackOverflow Dataset

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
data_dir = './data'

mli.download_file(source=data_url, filepath="./data/stack_overflow_16k.tar.gz")
mli.extract_tgz(filepath='./data/stack_overflow_16k.tar.gz', folder='./data')

Download of https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz completed.
Extraction of ./data/stack_overflow_16k.tar.gz completed.


## Inspect StackOverflow Dataset

In [4]:
mli.list_dir_summary(data_dir)

There are 2 directories, and 2 in ./data
There are 4 directories, and 0 in ./data/test
There are 0 directories, and 2000 in ./data/test/python
There are 0 directories, and 2000 in ./data/test/java
There are 0 directories, and 2000 in ./data/test/csharp
There are 0 directories, and 2000 in ./data/test/javascript
There are 4 directories, and 0 in ./data/train
There are 0 directories, and 2000 in ./data/train/python
There are 0 directories, and 2000 in ./data/train/java
There are 0 directories, and 2000 in ./data/train/csharp
There are 0 directories, and 2000 in ./data/train/javascript


In [5]:
train_dir = f"{data_dir}/train"
test_dir = f"{data_dir}/test"

# Show contents of a random file
random_file = random.choice(os.listdir(f"{train_dir}/java"))
with open(f"{data_dir}/train/java/{random_file}") as f:
  print(f.read())

"exception in thread ""main"" blank.lang i've been training a little with blank and i wanted to make this partial program about random numbers in a matrix 3x5, where every number in a column may belong to a different dozen compared to others...i'll make a better explanation: ..in one line you can have 9 and 11 but you can't have 11 and 13 because they belong to the same dozen (the 11-20 one). people who know the tombola game will understand it...but the main error after compiling and executing with cmd, was the one below, the exception in thread main blank.lang...so i hope you can help me: afterall i'm a new blank programmer....  exception in thread ""main"" blank.lang.indexoutofboundsexception: index.  0 out-of-bounds for length 0.          at blank.base/jdk.internal.util.preconditions.outofbounds(unknown source).          at blank.base/jdk.internal.util.preconditions.outofboundscheckindex(unknown.  source).          at blank.base/jdk.internal.util.preconditions.checkindex(unknown sou

## Load StackOverflow Dataset

In [6]:
# Create the training and validation datasets, whereas the training set is 80% of the data.

batch_size = 32
seed = 42

raw_train_ds, raw_validation_ds  = utils.text_dataset_from_directory(
    directory=train_dir,
    batch_size=batch_size,
    seed=seed,
    validation_split=0.2,
    subset='both' # return both the train and validation set.
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Using 1600 files for validation.
Metal device set to: Apple M1 Max


In [7]:
# Sample the training set

for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print(f"Label: {label_batch.numpy()[i]}")
        print(f"Question: {text_batch.numpy()[i]}")
        print("----")

Label: 1
Question: b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can be easily fixed, please forgive me. my program has a tester class with a main. when i send that to my regularpolygon class, it sends it to the wrong constructor. i have two constructors. 1 without perameters..public regularpolygon().    {.       mynumsides = 5;.       mysidelength = 30;.    }//end default constructor...and my second, with perameters. ..public regularpolygon(int numsides, double sidelength).    {.        mynumsides = numsides;.        mysidelength = sidelength;.    }// end constructor...in my tester class i have these two lines:..regularpolygon shape = new regularpolygon(numsides, sidelength);.        shape.menu();...numsides and sidelength were declared and initialized earlier in the testing class...so what i want to happen, is the tester class sends numsides and sidelength to the second constructor and use it in that class. but it only uses the def

2023-07-17 09:40:44.272571: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
# Get the label information
class_names = raw_train_ds.class_names
for i, class_name in enumerate(class_names):
    print(f"Label {i} corresponds {class_name}")

Label 0 corresponds csharp
Label 1 corresponds java
Label 2 corresponds javascript
Label 3 corresponds python


In [9]:
# Load the test data
raw_test_ds = utils.text_dataset_from_directory(
    directory=test_dir,
    batch_size=batch_size
)

Found 8000 files belonging to 4 classes.


## Prepare the StackOverflow dataset for training

In this step we will:

1. Standardize - Involves removing punctuation, HTML elements, etc...
2. Tokenize - Splitting strings up into individual words (or characters, ngrams)
3. Vectorize - Convert words into numbers

### Build 'binary' vectorization mode to build a bag-of-words model.

In [10]:
VOCAB_SIZE = 10_000 # Maximum number of words in our vocabulary.
binary_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    output_mode="binary"
)

### build 'int' mode with a 1D ConvNet.

In [11]:
MAX_SEQUENCE_LENGTH = 250 # explicit maximum sequence length, which will cause the layer to pad or truncate sequences to exactly output_sequence_length

int_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    output_mode="int",
    output_sequence_length=MAX_SEQUENCE_LENGTH
)

In [12]:
# Adapt the text vectorizers to the data (build up the vocabulary)
# IMPORTANT: Only adapt on training data and not the test-data. Otherwise it would leak test data into the training data.
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [13]:
# Print the results of using these layers to preprocess data:
def binary_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [14]:
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print(f"Label: {first_label}")
print(f"Question: {first_question}")

Label: 2
Question: b'"what is the difference between these two ways to create an element? var a = document.createelement(\'div\');..a.id = ""mydiv"";...and..var a = document.createelement(\'div\').id = ""mydiv"";...what is the difference between them such that the first one works and the second one doesn\'t?"\n'


In [15]:
# Show the output created when using 'binary' (bag-of-words) model. As you can see we get a total of 10.000 values for the sentence.
print("'binary' vectorized question:", binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 0. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [16]:
# Show the output created using 'int' model.
_ = int_vectorize_text(first_question, first_label)[0]
print("'int' vectorized question:", _)

'int' vectorized question: tf.Tensor(
[[ 55   6   2 410 211 229 121 895   4 124  32 245  43   5   1   1   5   1
    1   6   2 410 211 191 318  14   2  98  71 188   8   2 199  71 178   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0


## Re-create the text using the 'int' TextVectorizer

Some things to point out:

- Punctuation and markup (HTML for example) are all gone
- [UNK] are used to donate words that are not part of the vocabulary, for example the word 'regularpolygon' is not part of the vocabulary.
- Some weird words are in the text for example 'systemoutprintlnwelcome' which actually was code: 'System.out.println("Welcome").
- The length of the text has been cut-off at 250 words or when the text was shorter the tensor was filled out to a maximum of 250 words. This was part of the configuration of the 'int' TextVectorizer.

In [17]:
# Re-create the text from the TextVectorizer
" ".join([int_vectorize_layer.get_vocabulary()[val] for val in tf.squeeze(_)])

'what is the difference between these two ways to create an element var a [UNK] [UNK] a [UNK] [UNK] is the difference between them such that the first one works and the second one doesnt                                                                                                                                                                                                                       '

In [18]:
# Apply both TextVectorizers on the training, validation and test data.
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_validation_ds = raw_validation_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_validation_ds = raw_validation_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

## Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

1) Dataset.cache keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.
2) Dataset.prefetch overlaps data preprocessing and model execution while training. You can learn more about both methods, as well as how to cache data to disk in the Prefetching section of the Better performance with the tf.data API guide.

In [19]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
    return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [20]:
# Confiure all datasets to use caching with autotune buffer size
binary_train_ds = configure_dataset(binary_train_ds)
binary_validation_ds = configure_dataset(binary_validation_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_validation_ds = configure_dataset(int_validation_ds)
int_test_ds = configure_dataset(int_test_ds)

## Train the model

In [21]:
binary_model = tf.keras.Sequential([
    layers.Dense(units=4)
])

# from_logits = True means that the last layer in the model does not have a softmax function (probability between 0 and 1 distributed over all the possible outcomes)
# from_logits = False means that the last layer in the model does have a softmax function (probability between 0 and 1 distributed over all the possible outcomes)
binary_model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                     optimizer=optimizers.Adam(),
                     metrics=["accuracy"])

history = binary_model.fit(binary_train_ds, validation_data=binary_validation_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 2s 7ms/step - loss: 1.1220 - accuracy: 0.6447 - val_loss: 0.9168 - val_accuracy: 0.7844
Epoch 2/10
200/200 [==============================] - 1s 6ms/step - loss: 0.7805 - accuracy: 0.8198 - val_loss: 0.7518 - val_accuracy: 0.7987
Epoch 3/10
200/200 [==============================] - 1s 5ms/step - loss: 0.6286 - accuracy: 0.8620 - val_loss: 0.6658 - val_accuracy: 0.8100
Epoch 4/10
200/200 [==============================] - 1s 5ms/step - loss: 0.5350 - accuracy: 0.8859 - val_loss: 0.6122 - val_accuracy: 0.8231
Epoch 5/10
200/200 [==============================] - 1s 5ms/step - loss: 0.4688 - accuracy: 0.9048 - val_loss: 0.5754 - val_accuracy: 0.8325
Epoch 6/10
200/200 [==============================] - 1s 6ms/step - loss: 0.4185 - accuracy: 0.9159 - val_loss: 0.5486 - val_accuracy: 0.8394
Epoch 7/10
200/200 [==============================] - 1s 6ms/step - loss: 0.3782 - accuracy: 0.9281 - val_loss: 0.5284 - val_accuracy: 0.8400
Epoch 

In [22]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=64, mask_zero=True),
        layers.Conv1D(filters=64, kernel_size=5, padding='valid', activation='relu', strides=2),
        layers.GlobalMaxPooling1D(),
        layers.Dense(units=num_labels)
    ])
    return model

In [23]:
# `vocab_size` is `VOCAB_SIZE + 1` since `0` is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=len(class_names))
int_model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
                  optimizer=optimizers.Adam(),
                  metrics=["accuracy"])
history = int_model.fit(int_train_ds, validation_data=int_validation_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 6s 30ms/step - loss: 1.1573 - accuracy: 0.5050 - val_loss: 0.7585 - val_accuracy: 0.6913
Epoch 2/5
200/200 [==============================] - 3s 14ms/step - loss: 0.6304 - accuracy: 0.7581 - val_loss: 0.5517 - val_accuracy: 0.7962
Epoch 3/5
200/200 [==============================] - 2s 12ms/step - loss: 0.3825 - accuracy: 0.8798 - val_loss: 0.4813 - val_accuracy: 0.8206
Epoch 4/5
200/200 [==============================] - 2s 12ms/step - loss: 0.2131 - accuracy: 0.9469 - val_loss: 0.4728 - val_accuracy: 0.8219
Epoch 5/5
200/200 [==============================] - 2s 12ms/step - loss: 0.1070 - accuracy: 0.9805 - val_loss: 0.4947 - val_accuracy: 0.8188


## Compare the two models

In [24]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 40004     
                                                                 
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          640064    
                                                                 
 conv1d (Conv1D)             (None, None, 64)          20544     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                                 
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


## Evaluate both models

In [26]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print(f"Binary model accuracy: {binary_accuracy:2.2%}")
print(f"Int model accuracy: {int_accuracy:2.2%}")

250/250 [==============================] - 1s 5ms/step - loss: 0.5138 - accuracy: 0.8124
Binary model accuracy: 81.64%
Int model accuracy: 81.24%


## Export the model

There is a performance difference to keep in mind when choosing where to apply tf.keras.layers.TextVectorization.

Using it outside of your model enables you to do asynchronous CPU processing and buffering of your data when training on GPU.

So, if you're training your model on the GPU, you probably want to go with this option to get the best performance while developing your model, then switch to including the TextVectorization layer inside your model when you're ready to prepare for deployment.

In [27]:
export_model = tf.keras.Sequential(
    [binary_vectorize_layer, binary_model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

# Test it with `raw_test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(raw_test_ds)
print("Accuracy: {accuracy:2.2%}")

250/250 [==============================] - 2s 9ms/step - loss: 0.5176 - accuracy: 0.8164
Accuracy: {accuracy:2.2%}


In [28]:
def get_string_labels(predicted_scores_batch):
  predicted_int_labels = tf.math.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(raw_train_ds.class_names, predicted_int_labels)
  return predicted_labels

## Run inference on new data

In [29]:
inputs = [
    "how do I extract keys from a dict into a list?",  # 'python'
    "debug public static void main(string[] args) {...}",  # 'java'
]

predicted_scores = export_model.predict(inputs)
predicted_labels = get_string_labels(predicted_scores)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

1/1 [==============================] - 0s 194ms/step
Question:  how do I extract keys from a dict into a list?
Predicted label:  b'python'
Question:  debug public static void main(string[] args) {...}
Predicted label:  b'java'


In [30]:
predicted_scores

array([[0.36900595, 0.2443902 , 0.21475226, 0.8020787 ],
       [0.7013092 , 0.782961  , 0.08905066, 0.08734852]], dtype=float32)

# Example 2: Predict the author of Iliad translations

In [32]:
DIRECTORY_URL = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
FILE_NAMES = ['cowper.txt', 'derby.txt', 'butler.txt']

for name in FILE_NAMES:
    mli.download_file(f"{DIRECTORY_URL}{name}", f'./data2/{name}')

mli.list_dir_summary('./data2')

Download of https://storage.googleapis.com/download.tensorflow.org/data/illiad/cowper.txt completed.
Download of https://storage.googleapis.com/download.tensorflow.org/data/illiad/derby.txt completed.
Download of https://storage.googleapis.com/download.tensorflow.org/data/illiad/butler.txt completed.
There are 0 directories, and 3 in ./data2


## Load the dataset

In [33]:
def labeler(example, index):
    return example, tf.cast(index, tf.int64)

In [38]:
labeled_data_sets = []

for i, filename in enumerate(FILE_NAMES):
    lines_dataset = tf.data.TextLineDataset(filenames=f"./data2/{filename}")
    labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
    labeled_data_sets.append(labeled_dataset)

## Next, combine these labeled datasets into a single dataset using Dataset.concatenate, and shuffle it with Dataset.shuffle

In [37]:
BUFFER_SIZE = 50_000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [39]:
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

## Inspect some examples

In [42]:
for text, label in all_labeled_data.take(10):
  print("Sentence: ", text.numpy())
  print("Label:", label.numpy())

Sentence:  b'"Tydides, nor exaggerated praise'
Label: 1
Sentence:  b'criers round to call the people in assembly; so they called them and'
Label: 2
Sentence:  b'This by the son of Telamon was borne'
Label: 1
Sentence:  b"heaven's name, do I now see? Is it not Hector come to life again? Every"
Label: 2
Sentence:  b"Alighting, thus his speech to Priam turn'd."
Label: 0
Sentence:  b'bones be laid apart from yours, Achilles, but with them; even as we'
Label: 2
Sentence:  b'To face my spear; of most unhappy sires'
Label: 1
Sentence:  b"Who sits in sorrow by the high-prow'd ships,"
Label: 1
Sentence:  b'Led to the city consecrate to Mars'
Label: 0
Sentence:  b'sorely against my will. Of all inhabited cities under the sun and stars'
Label: 2


## Prepare the dataset for training (https://www.tensorflow.org/tutorials/load_data/text#prepare_the_dataset_for_training_2)

In [43]:
tokenizer = tf_text.UnicodeScriptTokenizer()

In [44]:
def tokenize(text, unused_label):
  lower_case = tf_text.case_fold_utf8(text)
  return tokenizer.tokenize(lower_case)

In [49]:
tokenized_ds = all_labeled_data.map(tokenize)

## Inspect some samples after the tokenization

In [50]:
for text_batch in tokenized_ds.take(5):
  print("Tokens: ", text_batch.numpy())

Tokens:  [b'"' b'tydides' b',' b'nor' b'exaggerated' b'praise']
Tokens:  [b'criers' b'round' b'to' b'call' b'the' b'people' b'in' b'assembly' b';'
 b'so' b'they' b'called' b'them' b'and']
Tokens:  [b'this' b'by' b'the' b'son' b'of' b'telamon' b'was' b'borne']
Tokens:  [b'heaven' b"'" b's' b'name' b',' b'do' b'i' b'now' b'see' b'?' b'is'
 b'it' b'not' b'hector' b'come' b'to' b'life' b'again' b'?' b'every']
Tokens:  [b'alighting' b',' b'thus' b'his' b'speech' b'to' b'priam' b'turn' b"'"
 b'd' b'.']


## Build a vocabulary by sorting tokens by frequency and keeping the top VOCAB_SIZE tokens

In [55]:
tokenized_ds = configure_dataset(tokenized_ds)

vocab_dict = collections.defaultdict(lambda: 0)
for toks in tokenized_ds.as_numpy_iterator():
  for tok in toks:
    vocab_dict[tok] += 1

# Code above results in a dictionary of 14.262 tokens, now sort the dictionaries
# based on the number of occurances column.

vocab = sorted(vocab_dict.items(), key=lambda x: x[1], reverse=True)
vocab = [token for token, count in vocab]
vocab = vocab[:VOCAB_SIZE]
vocab_size = len(vocab)
print("Vocab size: ", vocab_size)
print("First five vocab entries:", vocab[:5])

Vocab size:  10000
First five vocab entries: [b',', b'the', b'and', b"'", b'of']


## Convert the tokens into integers.

Use the vocab set to create a tf.lookup.StaticVocabularyTable. You will map tokens to integers in the range [2, vocab_size + 2]. As with the TextVectorization layer, 0 is reserved to denote padding and 1 is reserved to denote an out-of-vocabulary (OOV) token.

In [61]:
keys = vocab
values = range(2, len(vocab) + 2)  # Reserve `0` for padding, `1` for OOV tokens.

init = tf.lookup.KeyValueTensorInitializer(
    keys, values, key_dtype=tf.string, value_dtype=tf.int64)

num_oov_buckets = 1 # num_oov_buckets = number of buvkets to use for out-of-vocabulary keys.
vocab_table = tf.lookup.StaticVocabularyTable(initializer=init, num_oov_buckets=num_oov_buckets)

## Define a function to standardize, tokenize and vectorize the dataset using the tokenizer and lookup table

In [65]:
def preprocess_text(text: str, label: str):
  standardized = tf_text.case_fold_utf8(text)
  tokenized = tokenizer.tokenize(standardized)
  vectorized = vocab_table.lookup(tokenized)
  return vectorized, label

In [66]:
example_text, example_label = next(iter(all_labeled_data))
print("Sentence: ", example_text.numpy())
vectorized_text, example_label = preprocess_text(example_text, example_label)
print("Vectorized sentence: ", vectorized_text.numpy())

Sentence:  b'"Tydides, nor exaggerated praise'
Vectorized sentence:  [  93  610    2   64 9791 1033]


## Run the preprocess function on the dataset using Dataset.map

In [67]:
all_encoded_data = all_labeled_data.map(preprocess_text)

## Split the dataset into training and test sets

The Keras TextVectorization layer also batches and pads the vectorized data. Padding is required because the examples inside of a batch need to be the same size and shape, but the examples in these datasets are not all the same size—each line of text has a different number of words.

tf.data.Dataset supports splitting and padded-batching datasets:

In [68]:
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE)

In [70]:
train_data = train_data.padded_batch(BATCH_SIZE)
validation_data = validation_data.padded_batch(BATCH_SIZE)

## Inspect train_data and validation_data

validation_data and train_data are not collections of (example, label) pairs, but collections of batches. Each batch is a pair of (many examples, many labels) represented as arrays.

In [76]:
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Text batch shape:  (64, 20)
Label batch shape:  (64,)
First text example:  tf.Tensor(
[  93  610    2   64 9791 1033    0    0    0    0    0    0    0    0
    0    0    0    0    0    0], shape=(20,), dtype=int64)
First label example:  tf.Tensor(1, shape=(), dtype=int64)


## Adjust vocab_size due to padding.

Since you use 0 for padding and 1 for out-of-vocabulary (OOV) tokens, the vocabulary size has increased by two

In [77]:
vocab_size += 2

## Add prefetch and caching to the datasets (function from Example 1)

In [78]:
train_data = configure_dataset(train_data)
validation_data = configure_dataset(validation_data)

## Train the model

In [80]:
model = create_model(vocab_size=vocab_size, num_labels=len(FILE_NAMES))

model.compile(loss=losses.SparseCategoricalCrossentropy(),
              optimizer=optimizers.Adam(),
              metrics=["accuracy"])

history = model.fit(train_data, validation_data=validation_data, epochs=3)

Epoch 1/3
697/697 [==============================] - 28s 17ms/step - loss: 0.7943 - accuracy: 0.6292 - val_loss: 0.6155 - val_accuracy: 0.7314
Epoch 2/3
697/697 [==============================] - 8s 11ms/step - loss: 0.4903 - accuracy: 0.7997 - val_loss: 0.5808 - val_accuracy: 0.8140
Epoch 3/3
697/697 [==============================] - 7s 11ms/step - loss: 0.3634 - accuracy: 0.8724 - val_loss: 0.8057 - val_accuracy: 0.8170


In [81]:
loss, accuracy = model.evaluate(validation_data)

print(f"Loss: {loss}")
print(f"Accuracy: {accuracy:2.2%}")

79/79 [==============================] - 0s 6ms/step - loss: 0.8057 - accuracy: 0.8170
Loss: 0.8056844472885132
Accuracy: 81.70%


## Export the model

To make the model capable of taking raw strings as input, you will create a Keras TextVectorization layer that performs the same steps as your custom preprocessing function. Since you have already trained a vocabulary, you can use TextVectorization.set_vocabulary (instead of TextVectorization.adapt), which trains a new vocabulary.

In [82]:
preprocess_layer = layers.TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_text.case_fold_utf8,
    split=tokenizer.tokenize,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

preprocess_layer.set_vocabulary(vocab)

In [85]:
export_model = tf.keras.Sequential(
    [preprocess_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=optimizers.Adam(),
    metrics=['accuracy'])

In [87]:
# Create a test dataset of raw strings.
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = configure_dataset(test_ds)

loss, accuracy = export_model.evaluate(test_ds)

print(f"Loss: {loss}", loss)
print(f"Accuracy: {accuracy:2.2%}")

79/79 [==============================] - 3s 28ms/step - loss: 0.9030 - accuracy: 0.7360
Loss: 0.9029501080513 0.9029501080513
Accuracy: 73.60%


## Run inference on new data

In [89]:
inputs = [
    "Join'd to th' Ionians with their flowing robes,",  # Label: 1
    "the allies, and his armour flashed about him so that he seemed to all",  # Label: 2
    "And with loud clangor of his arms he fell.",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.math.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label.numpy())

1/1 [==============================] - 0s 46ms/step
Question:  Join'd to th' Ionians with their flowing robes,
Predicted label:  1
Question:  the allies, and his armour flashed about him so that he seemed to all
Predicted label:  2
Question:  And with loud clangor of his arms he fell.
Predicted label:  0


# Download more datasets using TensorFlow Datasets (TFDS, IMDB)

You can download many more datasets from TensorFlow Datasets.

In this example, we will use the IMDB Large Movie Review dataset to train a model for sentiment classification

In [90]:
# Training set.
train_ds = tfds.load(
    'imdb_reviews',
    split='train[:80%]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

2023-07-17 12:16:18.843986: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /Users/sodeso/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG4NDB8/imdb_reviews-test.t…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /Users/sodeso/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteG4NDB8/imdb_reviews-unsupe…

Dataset imdb_reviews downloaded and prepared to /Users/sodeso/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [91]:
# Validation set.
val_ds = tfds.load(
    'imdb_reviews',
    split='train[80%:]',
    batch_size=BATCH_SIZE,
    shuffle_files=True,
    as_supervised=True)

In [92]:
for review_batch, label_batch in val_ds.take(1):
  for i in range(5):
    print("Review: ", review_batch[i].numpy())
    print("Label: ", label_batch[i].numpy())

Review:  b"Instead, go to the zoo, buy some peanuts and feed 'em to the monkeys. Monkeys are funny. People with amnesia who don't say much, just sit there with vacant eyes are not all that funny.<br /><br />Black comedy? There isn't a black person in it, and there isn't one funny thing in it either.<br /><br />Walmart buys these things up somehow and puts them on their dollar rack. It's labeled Unrated. I think they took out the topless scene. They may have taken out other stuff too, who knows? All we know is that whatever they took out, isn't there any more.<br /><br />The acting seemed OK to me. There's a lot of unfathomables tho. It's supposed to be a city? It's supposed to be a big lake? If it's so hot in the church people are fanning themselves, why are they all wearing coats?"
Label:  0
Review:  b'Well, was Morgan Freeman any more unusual as God than George Burns? This film sure was better than that bore, "Oh, God". I was totally engrossed and LMAO all the way through. Carrey was

2023-07-17 12:17:12.856318: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Prepare the dataset for training

In [93]:
vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

# Make a text-only dataset (without labels), then call `TextVectorization.adapt`.
train_text = train_ds.map(lambda text, labels: text)
vectorize_layer.adapt(train_text)

In [94]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [95]:
train_ds = train_ds.map(vectorize_text)
val_ds = val_ds.map(vectorize_text)

In [99]:
# Configure datasets for performance as before (caching and prefetching)
train_ds = configure_dataset(train_ds)
val_ds = configure_dataset(val_ds)

In [100]:
model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=1)
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          640064    
                                                                 
 conv1d_2 (Conv1D)           (None, None, 64)          20544     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 660,673
Trainable params: 660,673
Non-trainable params: 0
_________________________________________________________________


In [101]:
model.compile(
    loss=losses.BinaryCrossentropy(from_logits=True), # from_logits is True since we don't have an activation function on last layer.
    optimizer=optimizers.Adam(),
    metrics=['accuracy'])

In [102]:
history = model.fit(train_ds, validation_data=val_ds, epochs=3)

Epoch 1/3
313/313 [==============================] - 9s 29ms/step - loss: 0.5376 - accuracy: 0.6679 - val_loss: 0.3741 - val_accuracy: 0.8332
Epoch 2/3
313/313 [==============================] - 5s 17ms/step - loss: 0.2983 - accuracy: 0.8674 - val_loss: 0.3213 - val_accuracy: 0.8588
Epoch 3/3
313/313 [==============================] - 5s 17ms/step - loss: 0.1819 - accuracy: 0.9306 - val_loss: 0.3273 - val_accuracy: 0.8640


In [103]:
loss, accuracy = model.evaluate(val_ds)

print(f"Loss: {loss}", loss)
print(f"Accuracy: {accuracy:2.2%}")

79/79 [==============================] - 0s 5ms/step - loss: 0.3273 - accuracy: 0.8640
Loss: 0.3272736668586731 0.3272736668586731
Accuracy: 86.40%


In [104]:
export_model = tf.keras.Sequential(
    [vectorize_layer, model,
     layers.Activation('sigmoid')])

export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False), # from_logits = False since we have sigmoid activation as last layer.
    optimizer=optimizers.Adam(),
    metrics=['accuracy'])

In [105]:
# 0 --> negative review
# 1 --> positive review
inputs = [
    "This is a fantastic movie.", # 1
    "This is a bad movie.", # 0
    "This movie was so bad that it was good.", # 1
    "I will never say yes to watching this movie.", # 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = [int(round(x[0])) for x in predicted_scores]

for input, label in zip(inputs, predicted_labels):
  print("Question: ", input)
  print("Predicted label: ", label)

1/1 [==============================] - 0s 75ms/step
Question:  This is a fantastic movie.
Predicted label:  1
Question:  This is a bad movie.
Predicted label:  0
Question:  This movie was so bad that it was good.
Predicted label:  0
Question:  I will never say yes to watching this movie.
Predicted label:  1
